In [1]:
import pyglet
import time
import datetime
from random import randint
import random
import numpy as np
import socket

In [2]:
def make_sprite(img, window, center = 1):
    sprite = pyglet.sprite.Sprite(img)
    if center:
        sprite.x = (window.width - sprite.width)//2
        sprite.y = (window.height - sprite.height)//2
    return sprite   

In [25]:
####################################################
fileroot = 'E:\\Diamond\\own_expo\\'
filewrite = open(fileroot + 'record.txt','w')
filewrite.write('')
filewrite.close()


file_cross = open(fileroot + 'cross_sign.txt','w')
file_cross.write('0')
file_cross.close()

endgame = open(fileroot + 'endgame.txt','w')
endgame.write('0')
endgame.close()


filewrite = open(fileroot + 'record.txt','a')
############################################################
#

### NOW START THE SOUND PROGRAM ########

In [26]:
#create sequences of labels
C_OVR  = [0,1] #MI classes, [0,1,2,3] for left hand, right hand, feet, tongue
_classes  = C_OVR*7 #*trials per MI class
random.shuffle(_classes) #randomize sequence of MI classes
# index of the trial
n = 0



#########################  Connect to Openvibe aquisition server (AS) ########################################################
# host and port of tcp tagging server
HOST = '127.0.0.1' #local machine address
PORT = 15361 #port to connect to AS

# transform a value into an array of byte values in little-endian order.
def to_byte(value, length):
    for x in range(length):
        yield value%256
        value//=256
        
# connect 
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((HOST, PORT))

#padding of zeros to keep length of tag consistant
padding=[0]*8

#############################################################################################################################



#used for timing, maybe not useful.
reso = 0.0005

t0 = datetime.datetime.now()
#when will cross/cue/rest appear next
cross_t = t0 + datetime.timedelta(0,randint(0,5)/10 + 6)
cue_t = cross_t + datetime.timedelta(0,1)
rest_t = cue_t + datetime.timedelta(0,4)

cross_exed = 0
cue_exed = 0
rest_exed = 0



window = pyglet.window.Window(fullscreen=True)

#load cues (and cross and rest) pictures and mak einto stripe objects
lh_img = pyglet.image.load('E:\\Diamond\\cues\\left_hand.png')
rh_img = pyglet.image.load('E:\\Diamond\\cues\\right_hand.png')
lh_anim = pyglet.image.load_animation('E:\\Diamond\\cues\\lh_fast2.gif')
rh_anim = pyglet.image.load_animation('E:\\Diamond\\cues\\rh_fast2.gif')
cross_img = pyglet.image.load('E:\\Diamond\\cues\\fixation.png')
rest_img = pyglet.image.load('E:\\Diamond\\cues\\black.png')
#lh = make_sprite(lh_img, window)
#rh = make_sprite(rh_img, window)
lh = make_sprite(lh_anim, window)
rh = make_sprite(rh_anim, window)
cross = make_sprite(cross_img, window)
rest = make_sprite(rest_img, window)

#default stripe to draw is rest (black)
stripe = rest

@window.event
def on_draw():
    window.clear()
    stripe.draw()
    
def close_window(dt):
    global window
    global s
    s.close()
    window.close()
    filewrite.close()

    pyglet.app.exit()
    print ('end')

def move2(dt):
    #have to mention that these are global variable or it wont work
    global n
    global stripe
    global cross_t
    global cue_t
    global rest_t
    global cross_exed
    global cue_exed
    global rest_exed
    global reso
    global s
    global padding
    
    #when time approaches nearer than thrshold (reso), draw this picture
    d_cross = (datetime.datetime.now() - cross_t).total_seconds()
    if  cross_exed == 0 and ((abs(d_cross) <= reso) or (d_cross>=reso)):
        stripe = cross
        
        filewrite.write('cross,' + str(datetime.datetime.now()) + '\n')
        window.flip()
        
        #tell the sound program to play the sound
        file_cross = open(fileroot + 'cross_sign.txt','w')
        file_cross.write('1')
        file_cross.close()
        
        print ('cross', datetime.datetime.now())
        print ('_classes', _classes[n])

        cross_exed = 1
        
        
    d_cue = (datetime.datetime.now() - cue_t).total_seconds()
    if  cue_exed == 0 and ((abs(d_cue) <= reso) or (d_cue>=reso)):
        #define which picture is which label
        if _classes[n] == 0:
            stripe = lh
            EVENT_ID = 0x441 #LEFT HAND (1089)
        elif _classes[n] == 1:
            stripe = rh
            EVENT_ID = 0x442 #RIGHT HAND (1090)
            
        #same time tag for record.txt and openvibe eeg streams
        time_datetime = datetime.datetime.now()
        time_unix = time.time()
        
        filewrite.write('cue,' + str(_classes[n]) + ',' + str(time_datetime) + '\n')
        window.flip()
        
        #event_id to byte format
        event_id=list(to_byte(EVENT_ID, 8))
        # timestamp can be either the posix time in ms, or 0 to let the acquisition server timestamp the tag itself.
        timestamp=list(to_byte(int(time_unix*1000), 8))
        print (timestamp)
        #send tag to openvibe, tag is padding + event_id + timestamp in unix time, in uint64 format
        s.sendall(bytearray(padding+event_id+timestamp))
        print ('cue', datetime.datetime.now())
        print ('_classes', _classes[n])

        cue_exed = 1
    

    d_rest = (datetime.datetime.now() - rest_t).total_seconds()
    if  rest_exed == 0 and ((abs(d_rest) <= reso) or (d_rest>=reso)):
        stripe = rest
        
        filewrite.write('rest,' + str(datetime.datetime.now()) + '\n')
        window.flip()
        print ('rest', datetime.datetime.now())
        print ('_classes', _classes[n])
        
        rest_exed = 1
        
     
    #next trial
    if cross_exed == 1 and cue_exed==1 and rest_exed == 1:
        cross_t = rest_t + datetime.timedelta(0,randint(0,5)/10 + 6)
        cue_t = cross_t + datetime.timedelta(0,1)
        rest_t = cue_t + datetime.timedelta(0,4)
        
        cross_exed = 0
        cue_exed = 0
        rest_exed = 0
        
        n = n +1
        if n >= len(_classes):
            pyglet.clock.schedule_once(close_window, 5)
            # tell the sound file to play the end sound
            endgame = open(fileroot + 'endgame.txt','w')
            endgame.write('1')
            endgame.close()
            
            
            print ('here end')
    
    
    

pyglet.clock.schedule_interval(move2, 1/1000.0)

pyglet.app.run() #manually close window to finish loop

cross 2019-10-15 16:57:10.271366
_classes 1
[85, 70, 236, 207, 109, 1, 0, 0]
cue 2019-10-15 16:57:11.270852
_classes 1
rest 2019-10-15 16:57:15.267746
_classes 1
cross 2019-10-15 16:57:21.365560
_classes 0
[171, 113, 236, 207, 109, 1, 0, 0]
cue 2019-10-15 16:57:22.364908
_classes 0
rest 2019-10-15 16:57:26.361954
_classes 0
cross 2019-10-15 16:57:32.758845
_classes 0
[45, 158, 236, 207, 109, 1, 0, 0]
cue 2019-10-15 16:57:33.758782
_classes 0
rest 2019-10-15 16:57:37.756263
_classes 0
cross 2019-10-15 16:57:43.869664
_classes 1
[148, 201, 236, 207, 109, 1, 0, 0]
cue 2019-10-15 16:57:44.869069
_classes 1
rest 2019-10-15 16:57:48.866494
_classes 1
cross 2019-10-15 16:57:55.263765
_classes 0
[22, 246, 236, 207, 109, 1, 0, 0]
cue 2019-10-15 16:57:56.263428
_classes 0
rest 2019-10-15 16:58:00.260409
_classes 0
cross 2019-10-15 16:58:06.758274
_classes 1
[252, 34, 237, 207, 109, 1, 0, 0]
cue 2019-10-15 16:58:07.757567
_classes 1
rest 2019-10-15 16:58:11.771248
_classes 1
cross 2019-10-15 16:5